## Importing libraries

In [1]:
#importing libraries
from fastai import *
from fastai.vision.all import *
from fastai.metrics import error_rate
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from pathlib import Path


C:\Users\rohit\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x  = 'dataset/seg_train'
path = Path(x)
path.ls()

(#6) [Path('dataset/seg_train/buildings'),Path('dataset/seg_train/forest'),Path('dataset/seg_train/glacier'),Path('dataset/seg_train/mountain'),Path('dataset/seg_train/sea'),Path('dataset/seg_train/street')]

In [5]:
np.random.seed(40)
data = ImageDataLoaders.from_folder(path, train = '.', valid_pct=0.2,
                                  ds_tfms=aug_transforms(), size=224,
                                  num_workers=4).normalize(imagenet_stats)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck
Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


AttributeError: normalize